In [4]:
import math
from model import EncoderCNN, DecoderRNN
from data_loader import get_loader
from data_loader_val import get_loader as val_get_loader
from pycocotools.coco import COCO
from torchvision import transforms
from tqdm.notebook import tqdm
import torch.nn as nn
import torch
import torch.utils.data as data
from collections import defaultdict
import json
import os
import sys
import numpy as np
from nlp_utils import clean_sentence, bleu_score

In [7]:
from data_loader import get_loader
from data_loader_val import get_loader as val_get_loader

#dataset dir path
cocoapi_dir = "cocoapi"
batch_size = 128 
vocab_threshold = 5
vocab_from_file = True
embed_size = 256
hidden_size = 512
num_epochs = 3
save_every = 1
print_every = 20
log_file = "training_log.txt"

transform_train = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            (0.485, 0.456, 0.406),
            (0.229, 0.224, 0.225),
        ),
    ]
)

#Build data loader.
data_loader = get_loader(
    transform = transform_train,
    mode="train",
    batch_size=batch_size,
    vocab_threshold = vocab_threshold,
    vocab_from_file=vocab_from_file,
    cocoapi_loc=cocoapi_dir,
)
#the size of the vocabulary
vocab_size = len(data_loader.dataset.vocab)
print("Vocab size is : ", vocab_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.48s)
creating index...
index created!
Obtaining caption lengths...


100%|████████████████████████████████████████████████████████████████| 591753/591753 [00:28<00:00, 20907.49it/s]


Vocab size is :  11543


In [9]:
import torch
import torch.nn as nn
import torchvision.models as models
#initializing the encoder and decoder
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

#move models to device
device = torch.device("mps")
encoder.to(device)
decoder.to(device)

/Users/satvikahuja13/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/satvikahuja13/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DecoderRNN(
  (embed): Embedding(11543, 256)
  (lstm): LSTM(256, 512, batch_first=True)
  (linear): Linear(in_features=512, out_features=11543, bias=True)
)